# Project Cryosphere by Matthias Göbel and Philipp Gregor
## Part 1

Matthias did exercises 1,3 and 5, while Philipp was responsible for exercises 2 and 4. The revision we did together.

In [ ]:
# imports and defaults
import pandas as pd  
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
pd.options.display.max_rows = 14
import seaborn as sns
sns.set_style('ticks')
sns.set_context('talk')
import scipy.constants as const
import itertools

In [ ]:
# read the data
df = pd.read_csv('data/aws_data_zhadang_localtime.csv', index_col=0, parse_dates=True)

In [ ]:
#Definition of used constants, according to section Appendix of the task
sigma = 5.67e-8  #Stefan-Boltzmann constant                 (W m⁻² K⁻⁴)
eps_ice = 1.0    #Emissivity of ice                         ()
c_p   = 1005.0   #Specific heat capacity of air             (J kg⁻¹ K⁻¹)
rho_0 = 1.29     #Density of air at mean sea level pressure (kg m⁻³)
p_0   = 1013.25  #Mean sea level pressure                   (hPa)
k     = 0.4      #Von Karman constant                       ()
z_m   = 2.5      #Measurement height of wind                (m)
z_h   = 2.0      #Measurement height of temperature         (m)
z_v   = 2.0      #Measurement height of humidity            (m)
z_0m  = 0.0017   #Surface roughness length for momentum     (m)
z_0h  = 0.0012   #Surface roughness length for temperature  (m)
z_0v  = 0.0012   #Surface roughness length for vapor        (m)
L_v   = 250.0e4  #Latent heat of evaporation                (J kg⁻¹)
L_s   = 283.0e4  #Latent heat of sublimation                (J kg⁻¹)
L_m   = 334.0e3  #Latent heat of Melt                       (J kg⁻¹)
T_0   = 273.15   #Triple point of water                     (K)
R_v   = 465.     #Gas constant of water vapor               (J kg⁻¹ K⁻¹)</div><i class="fa fa-lightbulb-o "></i>



## 1 Radiative Fluxes

In this first part, we look at the different longwave and shortwave contributions to the net radiative flux:
$ 
Q_{\mathrm{net}}=\mathrm{SW_{in}+SW_{out}+LW_{in}+LW_{out}}
$

### 1.1 Accuracy of the different sensors

As described in *Maussion et al. (2011)* three types of sensors (all from Campbell Scientific) were used to measure the radiation fluxes:
- NR-Lite for net radiation (http://s.campbellsci.com/documents/us/manuals/nr-lite.pdf)
- CS300 for incoming and outgoing shortwave radiation (https://s.campbellsci.com/documents/us/manuals/cs300.pdf)
- IRTS-P for outgoing longwave radiation (http://s.campbellsci.com/documents/us/manuals/irts.pdf)

The IRTS-P measures the outgoing longwave radiation and computes the surface temperature $T$ from it (with an emissivity of one) . We use Stefan-Boltzmann's law to recalculate the flux $F$ (again with an emissivity of 1). In this way we get the actual measured flux back, including an emissivity unequal 1.

$F=\sigma T^4$

where $\sigma$ is the Stefan-Boltzmann constant.


In [ ]:
df["LWOUT"]=-const.sigma*(df.SURFTEMP+273.15)**4  #convert surface temperature to Kelvin and calculate LWOUT with Stefan-Boltzmann

In the manuals we can find the accuracy of the three instruments. The absolute error of the IRTS-P is given in $^\circ \mathrm{C}$: $\Delta T=\pm 0.3 \,^\circ \mathrm{C}$. We can calculate the relative error of outgoing longwave radiation $F$ by using Stefan-Boltzmann's law and the variance formula for the propagation of uncertainty (https://en.wikipedia.org/wiki/Propagation_of_uncertainty):
$$\Delta F=\left|\frac{\partial F}{\partial T} \Delta T \right|=4F\frac{\Delta T}{T}$$
$$\Rightarrow \frac{\Delta F}{F}=4\frac{\Delta T}{T}$$
To estimate this expression we use the mean value of the surface temperature:


In [ ]:
dF=4*0.3/(df.SURFTEMP+273).mean()*100 #relative error in %
dF

The relative error is then only $\Delta F =\pm 0.5\%$. This is  however underestimated since the calculation of $F$ with Stefan-Boltzmann's law is not exact.

Accuracy of the instruments:
- NR-Lite:  
    - Directional error: $< 30 \,\mathrm{W m^{-2}}$(at an incidence angle of 0 - 60° and radiation $1000 \,\mathrm{W m^{-2}}$)
    - Sensor asymmetry: $\pm 5\%$ 
- CS300: $\pm 5\%$ 
- IRTS: >$\pm 0.5 \%$ 

### 1.2 Spectral ranges
In the manuals we also find the respective spectral ranges of the used instruments:
- NR-Lite: $\mathrm{0.2 - 100 \,\mu m}$
- CS300: $\mathrm{0.36 - 1.12 \,\mu m}$
- IRTS-P: $\mathrm{6 - 14 \,\mu m}$


Since the AWS was not equipped with a sensor measuring incoming longwave radiation, it has to be computed as a residual of the radiation budget equation. However, since the NR-Lite can measure lower wavelengths than the CS300 and higher wavelengths than the IRTS-P $\mathrm{SW_{in},\,SW_{out}\, and\,LW_{out}}$ will be underestimated. Thus, $\mathrm{LW_{in}}$ will be overestimated.

### 1.3 Analysing the different contributions

Having in mind that the spectral ranges of the different instruments are not matching, we now compute $\mathrm{LW_{in}}$ as the residual of the other fluxes and then plot the monthly averages of the different components.

In [ ]:
df.SWOUT=-df.SWOUT #correct sign 

In [ ]:
df["LWIN"]=df.NETRAD-df.SWIN-df.SWOUT-df.LWOUT #calculate LWIN as residual

Below we plot the monthly averages of the different components. We take the absolute values (except for net radiation) so that we can better compare the fluxes. 

In [ ]:
#To visualize the difference between individual days in one month we use the monthly standard deviation of daily average values as errorbars. In this way we disregard the diurnal cycle.

In [ ]:
df_monthly=df.resample('MS').mean()

for i in [["NETRAD","Net radiation"],
          ["SWIN","Incoming shortwave radiation"],["SWOUT","Outgoing shortwave radiation"],
          ["LWIN","Incoming longwave radiation"],["LWOUT","Outgoing longwave radiation"]]:
    plt.figure()
    d=df_monthly[i[0]]
    
    if i[0]!="NETRAD":
        d=abs(d) #use absolute values

    d.plot(marker="o",linestyle="")
    plt.xlabel("")
    plt.ylabel("Radiative flux ($\mathrm{Wm^{-2}}$)")
    plt.xlim("2010-09-01","2012-09-01")
    plt.suptitle(i[1])
    plt.grid()


In winter, from November to February, negative net radiation prevails, whereas in summer, from May to September, positive values prevail. In the transition months March, April and October both, positive and negative values occur almost equally often.

The other components and their respective driving processes will be addressed in the following section.

### 1.4 Driving processes

**Daily averages plot**

In [ ]:
df_daily=df.resample('D').mean().copy()
for i in [["NETRAD","Net radiation"],
          ["SWIN","Incoming shortwave radiation"],["SWOUT","Outgoing shortwave radiation"],
          ["LWIN","Incoming longwave radiation"],["LWOUT","Outgoing longwave radiation"]]:
    plt.figure()
    d=df_daily[i[0]]
    
    if i[0]!="NETRAD":
        d=abs(d) #use absolute values

    d.plot(marker=".",linestyle="")
    plt.xlabel("")
    plt.ylabel("Radiative flux ($\mathrm{Wm^{-2}}$)")
    plt.xlim("2010-09-01","2012-10-01")
    plt.suptitle(i[1])
    plt.grid()
    


#### Incoming shortwave radiation
Incoming shortwave radiation mainly follows the seasonal cycle of solar irradiance at the top of the atmosphere: Minimum in winter, maximum in summer. Direct and diffuse solar radiation are the main drivers. The solar irradiance at the TOA minus the radiation that is reflected or absorbed in the clear atmosphere gives the upper limit for the incoming shortwave radiation. The daily average plot above shows this quite well.

If clouds are present, incoming shortwave radiation is reduced. Especially in summer (June to September) solar irradiance is far below the upper limit on many days, as can be seen on the daily averages plot, again. Therefore the largest fluxes occur in May. 

Relative humidity can serve as a proxy for cloud cover. Below we plot incoming shortwave radiation versus relative humidity for the month of July (2011 and 2012). We take only one month to minimize the effect of the seasonal cycle (in July the seasonal cycle is also comparably weak). This leads to a correlation coefficient of -0.64. The higher relative humidity, the lower the incoming shortwave radiation, in general

The last and probably least important contribution is radiation that is reflected from the ground and then reflected again from clouds and radiation that is reflected from surrounding surfaces (ice and snow) and reaches the sensor from above. In this way incoming shortwave radiation can exceed the mentioned upper limit.

In [ ]:
df_mon=df_daily.loc[df_daily.index.month==7]
d1=df_mon["RH"]
d2=df_mon["SWIN"]
plt.scatter(d1,d2)
plt.ylabel("Incoming shortwave radiation ($\mathrm{Wm^{-2}}$)")
plt.xlabel("Relative humidity [%]")
corr=np.corrcoef(d1,d2)[0,1]
plt.text(min(d1),min(d2),"Correlation coefficient: {0}".format(np.round(corr,2)))
plt.suptitle("Correlation of $\mathrm{SW_{in}}$ and relative humidity in July");

#### Outgoing shortwave radiation
Outgoing shortwave radiation is the incoming shortwave radiation that is reflected from the surface. It depends on the value of the incoming shortwave radiation (and thus on seasonal cycle and clouds) and the albedo of the surface. 
To investigate the latter effect we plot the daily ratio of outgoing to incoming shortwave radiation below.

Since snow has a higher albedo than ice, this ratio is usually higher in winter than in summer. The fresher the snow, the higher the albedo is. To see the effect of snowfall, we plot vertical lines for days with a height increase of more than 1 mm or a measured precipitation at an air temperatures below freezing point. In the plot we can see that also in summer there is snowfall where the ratio  $\mathrm{SW_{out}/SW_{in}}$ increases rapidly.

The snowfall events (green vertical lines) coincide with an increase in daily albedo. However, not all albedo increases are captured in this way. This can be due to failure of the instruments (the SR50 sensor was not operating until beginning of July 2012 and the precipitation gauge was down from mid September 2011 to end of April 2012).


In [ ]:
(df_daily.SR50.shift()>df_daily.SR50+0.01).plot()
df_daily.SR50.plot();

In [ ]:
df_daily["SNOW"]=0
df_daily["MELTING"]=np.where(df_daily.SURFTEMP>-0.1,1,0)
df_daily["SNOW"]=np.where(((df_daily.PRCP>0.01) & (df_daily.TEMP<0)) | (df_daily["SR50"].shift()>df_daily["SR50"]+0.05) ,1,0)

In [ ]:
(-df_daily["SWOUT"]/df_daily["SWIN"]).plot()
plt.ylabel(" $\mathrm{SW_{out}/SW_{in}}$")
plt.xlabel("")
plt.suptitle("Daily albedo: $\mathrm{SW_{out}/SW_{in}}$, snowfall events (green vertical lines) and melting events (red vertical lines)");
plt.ylim(0,1)

for t in df_daily.index[df_daily["SNOW"]==1]:
    plt.axvline(x=t...,linewidth=1,color="g")


#### Incoming longwave radiation
The source of incoming longwave radiation is thermal radiation from the atmosphere and clouds. As before for the shortwave radiation we can easily see the effect of clouds in a scatter plot of incoming longwave radiation versus relative humidity. In August we get a correlation coefficient of 0.8.

The variability, however, is lower than for the shortwave fluxes, as we can see in the daily averages plot.
$\mathrm{LW_{out}}$ also shows a seasonal cycle, since the warmer atmosphere/clouds in summer radiate more energy away and the average cloud cover is higher in the summer months (cf. reduced incoming shortwave radiation on daily averages plot).


In [ ]:
df_mon=df_daily.loc[df_daily.index.month==8]
d1=df_mon["RH"]
d2=df_mon["LWIN"]
plt.scatter(d1,d2)
plt.ylabel("Incoming longwave radiation ($\mathrm{Wm^{-2}}$)")
plt.xlabel("Relative humidity [%]")
corr=np.corrcoef(d1,d2)[0,1]
plt.text(min(d1),min(d2),"Correlation coefficient: {0}".format(np.round(corr,2)))
plt.suptitle("Correlation of $\mathrm{LW_{in}}$ and relative humidity in August");

#### Outgoing longwave radiation


The outgoing longwave radiation depends on the temperature and emissivity of the surface.
The temperature dependence leads to the usual seasonal cycle. 

Emissivity depends on the surface characteristics. According to Kirchhoff's law the emissivity equals the absorptivity at a specific wavelength in thermal equilibrium. Since reflectivity (albedo) and absorptivity add up to one, we can say: The higher the albedo, the lower the emissivity.  

The scatter plot of $\mathrm{LW_{out}}$ versus $\mathrm{LW_{out}/LW_{in}}$ shows this anti-correlation quite clearly. Note that we still have the temperature effect included in this plot. As we only have the black body temperature of the surface available we cannot separate these two effects. Furthermore, we do not have thermal equilibrium here.


In [ ]:

d1=-df_daily["LWOUT"]/df_daily["LWIN"]
d2=-df_daily["LWOUT"]
plt.scatter(d1,d2)
plt.ylabel("Outgoing longwave radiation ($\mathrm{Wm^{-2}}$)")
plt.xlabel("$\mathrm{LW_{out}/LW_{in}}$")
corr=np.corrcoef(d1,d2)[0,1]
plt.text(min(d1),min(d2),"Correlation coefficient: {0}".format(np.round(corr,2)))
plt.suptitle("Correlation of $\mathrm{LW_{out}}$ and $\mathrm{LW_{out}/LW_{in}}$");

## 2 Turbulent Fluxes

TODO:
- References
- Check explanations

### 2.0 Defintions and Setup for Task 2

First, an additional column is added to the dataframe for SURFTEMP and TEMP in Kelvin, as values are needed in Kelvin by the functions in this chapter.

In [ ]:
df['SURFTEMP_K'] = df['SURFTEMP'] + T_0
df['TEMP_K'] = df['TEMP'] + T_0

Define the functions for latent and sensible heat flux in general (returning just a np.array)

In [ ]:
def calc_Q_H(T_s, T_a, p, v, z_m=z_m, z_0m=z_0m, z_h=z_h, z_0h=z_0h):
    """
    Calculates and returns sensible heat flux in W m⁻².
    
    T_s: surface temperature (K)
    T_a: air temperature (K)
    p: pressure (hPa)
    v: wind speed (m s⁻¹)
    
    For more information about the optional parameters z_m, ...
    please refer to the definition of constants at the beginning of this project
    
    p_0, rho_0, k, z_m, z_0m, z_h, z_0h are assumed to be defined beforehand.
    """
    x = c_p * rho_0 * p / p_0
    x = x * k**2 * v * (T_a - T_s)
    x = x / (np.log(z_m/z_0m) * np.log(z_h/z_0h))
    return x

In [ ]:
def get_L(T):
    """
    Gets the right value for L (L_v or L_s)
    based on the given temperature.
    Returns either a pandas DataFrame or a Numpy ndarray
    or a scalar based on the type of T
    
    T: temperature (K)
    """
    if(type(T) == pd.core.series.Series or type(T) == np.ndarray):
        L = T.copy()
        L[T >= T_0] = L_v
        L[T < T_0] = L_s
    else:
        if(T >= T_0):
            L = L_v
        else:
            L = L_s
    return L

In [ ]:
def calc_e_s(T):
    """
    Calculates and returns saturation vapor pressure in hPa
    for a given surface or air temperature T in Kelvin
    
    T: temperature (K)
    
    L_v, L_s, R_v are assumed to be defined beforehand.    
    """
    L = get_L(T)
    x = 6.11*np.exp(L/R_v * (1/T_0 - 1/T))
    return x

In [ ]:
def calc_e_a(T_a, RH):
    """
    Calculates and returns vapor pressure of air in hPa
    for a given air temperature T_a in Kelvin
    and relative humidity RH in percent.
    
    Internally uses calc_e_s to obtain
    the saturation vapor pressure for air at the given temperature
    
    T_a: air temperature (K)
    RH: relative humidity (%)
    
    L_v, L_s, R_v are assumed to be defined beforehand.   
    """
    x = calc_e_s(T_a)
    x = RH/100 * x
    return x

In [ ]:
def calc_Q_L(T_s, T_a, p, v, RH, z_m=z_m, z_0m=z_0m, z_v=z_v, z_0v=z_0v):
    """
    Calculates and returns latent heat flux in W m⁻².
    
    T_s: surface temperature (K)
    T_a: air temperature (K)
    p: pressure (hPa)
    v: wind speed (m s⁻¹)
    RH: relative humidity (%)
    
    
    For more information about the optional parameters z_m, ...
    please refer to the definition of constants at the beginning of this project
    
    L_v, L_s, R_v, p_0, rho_0, k, z_m, z_0m, z_v, z_0v are assumed to be defined beforehand.
    """
    L = get_L(T_a)
    
    x = 0.623 * L * rho_0 / p_0
    x = x * k**2 * v * (calc_e_a(T_a, RH) - calc_e_s(T_s))
    x = x / (np.log(z_m/z_0m) * np.log(z_v/z_0v))
    return x


In [ ]:
def calc_Q_HL_to_df(T_s, T_a, p, v, RH, z_m=z_m, z_0m=z_0m, z_h=z_h, z_0h=z_0h, z_v=z_v, z_0v=z_0v):
    """
    Calculates sensible and latent heat flux in W m⁻².
    Returns an indexed DataFrame with the two variables.
    
    T_s: surface temperature (K)
    T_a: air temperature (K)
    p: pressure (hPa)
    v: wind speed (m s⁻¹)
    RH: relative humidity (%)
    
    L_v, L_s, R_v, p_0, rho_0, k, z_m, z_0m, z_h, z_0h, z_v, z_0v are assumed to be defined beforehand.
    """
    flux_df = pd.DataFrame(index=T_s.index, columns=['Q_H', 'Q_L'])
    flux_df['Q_H'] = calc_Q_H(T_s, T_a, p, v, z_m=z_m, z_0m=z_0m, z_h=z_h, z_0h=z_0h)
    flux_df['Q_L'] = calc_Q_L(T_s, T_a, p, v, RH, z_m=z_m, z_0m=z_0m, z_v=z_v, z_0v=z_0v)
    return flux_df

In [ ]:
def calc_Q_HL_from_to_df(df, z_m=z_m, z_0m=z_0m, z_h=z_h, z_0h=z_0h, z_v=z_v, z_0v=z_0v):
    """
    Calculates sensible and latent heat flux in W m⁻²
    from the values contained in the given DataFrame.
    
    Returns a copy of the given DataFrame extended by the two variables.
    
    df: DataFrame with the needed data; these columns are used and required:
        SURFTEMP_K: surface temperature (K)
        TEMP_K: air temperature (K)
        PRESSURE: pressure(hPa)
        WINDSPEED: wind speed (m s⁻¹)
        RH: relative humidity(%)
        
    L_v, L_s, R_v, p_0, rho_0, k, z_m, z_0m, z_h, z_0h, z_v, z_0v are assumed to be defined beforehand.
    """
    flux_df = df.copy()
    flux_df['Q_H'] = calc_Q_H(df['SURFTEMP_K'], df['TEMP_K'], df['PRESSURE'], df['WINDSPEED'], z_m=z_m, z_0m=z_0m, z_h=z_h, z_0h=z_0h)
    flux_df['Q_L'] = calc_Q_L(df['SURFTEMP_K'], df['TEMP_K'], df['PRESSURE'], df['WINDSPEED'], df['RH'], z_m=z_m, z_0m=z_0m, z_v=z_v, z_0v=z_0v)
    return flux_df

### 2.1 Sensible heat flux and 2.2 Latent heat flux

In [ ]:
# Calculate fluxes based on standard values
df11 = calc_Q_HL_from_to_df(df)

In [ ]:
df11['Q_H'].plot();
h = plt.hlines(0, xmin=df11.index.min(), xmax=df11.index.max(), colors='k', linestyles='--', linewidth=0.5)
h.set_zorder(20)
plt.ylabel('$Q_H \: \mathrm{(W m^{-2})}$');
plt.xlabel('');
plt.title('Sensible heat flux');

$Q_H$ is most variable during the winter months and least variable during summer. In summer, the air temperature is often higher than the surface temperature and therefore the sensible heat flux is often positive, transporting energy into the ground. During winter, the difference between air and surface temperature is much more variable, depending e.g. on snowfall or clear sky conditions.

In [ ]:
e_a = calc_e_a(df['TEMP_K'], df['RH'])
e_s = calc_e_s(df['SURFTEMP_K'])

In [ ]:

#TO DO: leave out, as not referenced to it?
e_a.plot()
plt.title("Water vapor pressure")
plt.ylabel('$e_a \: (hPa)$')
plt.xlabel('');
plt.figure()
e_s.plot()
plt.title("Saturation vapor pressure")
plt.ylabel('$e_s \: (hPa)$')
plt.xlabel('');

In [ ]:
(e_a - e_s).plot()
plt.title("Difference of actual and saturation vapor pressure")
h = plt.hlines(0, xmin=df11.index.min(), xmax=df11.index.max(), colors='k', linestyles='--', linewidth=0.5)
h.set_zorder(20)
plt.ylabel('$e_a - e_s \: (hPa)$')
plt.xlabel('');

In [ ]:
df['RH'].plot();
plt.ylabel('RH (\%)');
plt.xlabel('');
plt.title('Relative humidity');

In [ ]:
df11['Q_L'].plot(c='g');
h = plt.hlines(0, xmin=df11.index.min(), xmax=df11.index.max(), colors='k', linestyles='--', linewidth=0.5)
h.set_zorder(20)
plt.ylabel('$Q_L \: \mathrm{(W m^{-2})}$');
plt.xlabel('');
plt.title('Latent heat flux');

The latent heat flux is most variable and more negative during the winter months. This is (see figures above) due to the big difference of vapor pressure in air compared to saturation vapor pressure. In winter, the actual vapor pressure is often up to 5 hPa smaller than the saturation vapor pressure, meaning that the conditions are very dry. These dry conditions (see also RH) cause a stronger latent heat flux (evaporation, sublimation) into the atmosphere, giving negative $Q_L$. During summer, the air is often closer to saturation, which results in less evaporation and sublimation and a smaller latent heat flux.

In [ ]:
df11_jan = df11.loc[df.index.month == 1]
jan_daily_cycle = df11_jan.groupby(df11_jan.index.hour).mean()
jan_daily_cycle.index = jan_daily_cycle.index + 0.5 #center time averages

In [ ]:
jan_daily_cycle['Q_H'].plot(label='$Q_H$');
jan_daily_cycle['Q_L'].plot(label='$Q_L$');
plt.xticks(np.arange(0,25,3));
plt.xlabel('Time (h)');
plt.ylabel('turbulent heat flux $\mathrm{(W m^{-2})}$');
plt.title('Sensible and latent heat flux - daily cycle of January');
plt.hlines(0, xmin=0, xmax=24, colors='k', linestyles='--', linewidth=0.5)
plt.legend();

In January, the latent heat flux is positive during nighttime. The ground cools by radiation during the night and therefore the cooler ground gains sensible heat from the warmer air. During daytime, the ground is heated by radiation and warmer than the air. Therefore the sensible heat flux is oriented from the ground into the atmosphere.
The latent heat flux is only slightly negative during night, as the ground is mainly frozen and (almost) only sublimation is possible. When heated during daytime, there is melting at the surface and more energy available for evaporation and sublimation. This results in a stronger latent heat flux to the rather dry January atmosphere.

In [ ]:
df11_jul = df11.loc[df.index.month == 7]
jul_daily_cycle = df11_jul.groupby(df11_jul.index.hour).mean()
jul_daily_cycle.index = jul_daily_cycle.index + 0.5

In [ ]:
jul_daily_cycle['Q_H'].plot(label='$Q_H$');
jul_daily_cycle['Q_L'].plot(label='$Q_L$');
plt.xticks(np.arange(0,25,3));
plt.xlabel('Time (h)');
plt.ylabel('Turbulent heat flux $\mathrm{(W m^{-2})}$');
plt.title('Sensible and latent heat flux - daily cycle of July');
plt.hlines(0, xmin=0, xmax=24, colors='k', linestyles='--', linewidth=0.5)
plt.legend();

In July, the air temperature is most of the time above 0°C, while the ground temperature of the glacier surface is typically around 0°C. This leads to a positive latent heat flux into the ground, especially during the time of the day with the highest air temperatures (around 14:00 to 15:00 local time).
The latent heat flux is always close to zero. This is most likely due to the very humid conditions, which suppress evaporation and sublimation.

### 2.3 Comparison to the other fluxes

In [ ]:
df13 = df11.resample('MS').mean()

In [ ]:
df13['Q_H'].plot(label='$Q_H$');
df13['Q_L'].plot(label='$Q_L$');
df13['NETRAD'].plot(label='net radiation');

plt.xlabel('');
plt.ylabel('Flux $\mathrm{(W m^{-2})}$');
plt.title('Sensible and latent heat flux vs net radiation flux - monthly averages');
plt.legend(loc="best");
h = plt.hlines(0, xmin=df11.index.min(), xmax=df11.index.max(), colors='k', linestyles='--', linewidth=0.5)


In general, the turbulent fluxes are of the same order of magnitude as the net radiation. Only during summer months, the net radiation exceeds the turbulent fluxes and these are less important. In contrast, the absolute value of each turbulent heat flux is larger than the net radiation during winter time. In general, the turbulent heat fluxes cannot be neglected.

### 2.4 Sensitivity to meteorological conditions

TO DO:
- Plot only relative difference?
- Divide into 2 plots, temperature and wind for better overview?
- other colors? hard to distinguish blue and violet

In [ ]:
wind_modifications = [-10, 0, 10] #Modification of the wind speed in percent
temp_modifications = [-1, 0, 1]   #Modification of the temperature in Kelvin

#Choose preferred variant: all combinations or just on parameter varying
#combined_modifications = list(itertools.product(wind_modifications, temp_modifications))
combined_modifications = [(-10, 0), (10, 0), (0, -1), (0, 1)]

In [ ]:
df14_Q_H = pd.DataFrame(index=df.index)
df14_Q_L = pd.DataFrame(index=df.index)
df14_Q_H_change = pd.DataFrame(index=df.index)
df14_Q_L_change = pd.DataFrame(index=df.index)
for mod in combined_modifications:
    label = (',%+3d \%%,%+dK') % mod
    windspeed = df['WINDSPEED'] * (1. + mod[0] / 100.)
    temp = df['TEMP_K'] + mod[1]
    df14_Q_H['$Q_{H%s}$'%label] = calc_Q_H(df['SURFTEMP_K'], temp, df['PRESSURE'], windspeed)#-df11.Q_H
    df14_Q_L['$Q_{L%s}$'%label] = calc_Q_L(df['SURFTEMP_K'], temp, df['PRESSURE'], windspeed, df['RH'])#-df11.Q_L
    
    df14_Q_H_change['$Q_{H%s}$'%label] = df14_Q_H['$Q_{H%s}$'%label] - df11.Q_H
    df14_Q_L_change['$Q_{L%s}$'%label] = df14_Q_L['$Q_{L%s}$'%label] - df11.Q_L

In [ ]:
flatui = ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
sns.set_palette(sns.color_palette("colorblind", 10));

In [ ]:
df14_Q_H_change.resample('MS').mean().plot();
h = plt.hlines(0, xmin=df14_Q_H.index.min(), xmax=df14_Q_H.index.max(), colors='k', linestyles='--', linewidth=0.5)
h.set_zorder(20)
plt.xlabel('');
plt.ylabel('Change in sensible heat flux $\mathrm{(W m^{-2})}$');
plt.title('Change in monthly averaged sensible heat fluxes with varying windspeed $\pm 10 \%$ and air temperature $\pm 1 K$');
plt.legend();

df14_Q_L_change.resample('MS').mean().plot();
h = plt.hlines(0, xmin=df14_Q_L.index.min(), xmax=df14_Q_L.index.max(), colors='k', linestyles='--', linewidth=0.5)
h.set_zorder(20)
plt.xlabel('');
plt.ylabel('Latent heat flux $\mathrm{(W m^{-2})}$');
plt.title('Change in monthly averaged latent heat fluxes with varying windspeed $\pm 10 \%$ and air temperature $\pm 1 K$');
plt.legend(loc="best");

In [ ]:
df14_Q_H.resample('MS').mean().plot();
h = plt.hlines(0, xmin=df14_Q_H.index.min(), xmax=df14_Q_H.index.max(), colors='k', linestyles='--', linewidth=0.5)
h.set_zorder(20)
plt.xlabel('');
plt.ylabel('Sensible heat flux $\mathrm{(W m^{-2})}$');
plt.title('Monthly averaged sensible heat fluxes with varying windspeed $\pm 10 \%$ and air temperature $\pm 1 K$');
plt.legend();

df14_Q_L.resample('MS').mean().plot();
h = plt.hlines(0, xmin=df14_Q_L.index.min(), xmax=df14_Q_L.index.max(), colors='k', linestyles='--', linewidth=0.5)
h.set_zorder(20)
plt.xlabel('');
plt.ylabel('Latent heat flux $\mathrm{(W m^{-2})}$');
plt.title('Monthly averaged latent heat fluxes with varying windspeed $\pm 10 \%$ and air temperature $\pm 1 K$');
plt.legend(loc="best");

Both, latent and sensible heat flux are by definition directly proportional to the windspeed. $Q_H$ has also a linear 
dependency on the air temperature and also $Q_L$ is dependent on the air temperature via the calculation of air vapor pressure from relative humidity.

Perturbing the wind speed leads according to the direct proportionality simply to a 10% change in the fluxes. As the temperature is perturbed by a constant value and the sensible heat flux depends on a temperature difference, the relative impact of the perturbation is largest, where the difference between surface and air temperature is the smallest. The air temperature perturbation is most noticeable for the latent heat flux where the flux is the largest.

These sensitivity experiments can be a good first hint, what happens if the temperature or wind changes or if there is a measurement error by a corresponding sensor. The approximate impact of changes in temperature or wind on the turbulent heat fluxes may be estimated this way. However, the atmosphere is a highly coupled system and changing one variable is often also connected to changes in other variables and processes, which again impact the turbulent fluxes. This has to be considered, when results from a sensitivity experiment are used for projecting the impacts of changes in one variable. E.g. increasing the wind speed will most probably also change the rate of evaporation/sublimation by changing the vapor pressure profile above ground. This means that also $e_a$ would adapt and result in a further change of the latent heat flux. This is not considered in this sensitivity experiment.

### 2.5 Sensitivity to the measurement heights

In [ ]:
df15 = df.copy()
df15['Q_H'] = calc_Q_H(df['SURFTEMP_K'], df['TEMP_K'], df['PRESSURE'], df['WINDSPEED'], z_m=z_m, z_h=z_h)
df15['Q_L'] = calc_Q_L(df['SURFTEMP_K'], df['TEMP_K'], df['PRESSURE'], df['WINDSPEED'], df['RH'], z_m=z_m, z_v=z_v)
df15['Q_H_low'] = calc_Q_H(df['SURFTEMP_K'], df['TEMP_K'], df['PRESSURE'], df['WINDSPEED'], z_m=(z_m-1), z_h=(z_h-1))
df15['Q_L_low'] = calc_Q_L(df['SURFTEMP_K'], df['TEMP_K'], df['PRESSURE'], df['WINDSPEED'], df['RH'], z_m=(z_m-1), z_v=(z_v-1))

In [ ]:
ax = df15.plot(y=['Q_H_low', 'Q_H']);
h = plt.hlines(0, xmin=df15.index.min(), xmax=df15.index.max(), colors='k', linestyles='--', linewidth=0.5)
h.set_zorder(20)
plt.ylabel('$Q_H$ $\mathrm{(W m^{-2})}$');
plt.xlabel('');
plt.title('Sensible heat flux');
plt.legend(['measurement height -1m', 'absolute measurement height'], loc='best');

df15.plot(y=['Q_L_low', 'Q_L'], label=['$Q_{L,low}$', '$Q_L$']);
h = plt.hlines(0, xmin=df15.index.min(), xmax=df15.index.max(), colors='k', linestyles='--', linewidth=0.5)
h.set_zorder(20)
plt.ylabel('$Q_L$ $\mathrm{(W m^{-2})}$');
plt.xlabel('');
plt.title('Latent heat flux');
plt.legend(['measurement height -1m', 'absolute measurement height'], loc='best');

As can be seen above and from the formula ($Q_H, Q_L \sim \ln(z_x / z_{0x})^{-1}$), the turbulent fluxes increase with lower measurement heights. This is logical, as turbulence and gradients in general increase towards the ground. This test shows the high uncertainty for the fluxes, as variations of approximately up to 70 $W m^{-2}$ are possible only due to this change in measurement height.

### 2.6 Roughness lengths

Roughness lengths are needed for the approximation of turbulent fluxes, However, they are hard to determine and very sensitive to a variety of influences. The roughness length for momentum on a glacier may vary widely e.g. between freshly fallen snow and a pure glacial ice surface. This problem is assessed also in the literature, which provides an idea of the range for the values of these roughness lengths.
Ambach (1986) assumes wind (momentum) roughness lengths $z_{0m}$ of $2 \cdot 10^{-3} m$ for snow and $1 \cdot 10^{-4} m$ for ice surfaces. This wide range seems very imprecise, but considering that both cases are realistic for a glacier, it simply shows the uncertainty based on temporal changes of the surface. For $z_{0h}$ (temperature) and $z_{0v}$ (water vapor) a value of $6 \cdot 10^{-6} m$ can be assumed (Ambach, 1986). Roughness lengths over an antarctic ice shelf were assessed in King et al. (1994). In their study they found a smaller roughness length for momentum of $z_{0m} \approx 5.6 \cdot 10^{-5} m$. Temperature and water vapor flux are again assumed to be equal and a value of $z_{0v} \approx z_{0h} \approx 5.1 \cdot 10^{-2} m$ is given.

There is a good summary about this topic in Hock (2005):

> *One problem concerns the specification of roughness lengths. They can
be derived from detailed measurements of wind, temperature and humidity profiles. However, these are seldom available and the roughness lengths are often estimated from published data (e.g., Greuell and Oerlemans, 1989; Konzelmann and Braithwaite, 1995). This poses a problem because the roughness lengths for wind reported over snow and ice vary by several orders of magnitude (summaries in Moore, 1983; Braithwaite, 1995a) ranging from 0.004 mm (Inoue, 1989) to 70 mm (Jackson and Carroll, 1978) over snow and from 0.003 mm (Antarctic blue ice
area; Bintanja and van den Broeke, 1995) to 120 mm (van den Broeke, 1996) over glacier ice. Generally, [$z_{0m}$]-values of a few mm tend to
be assumed in glacier applications. Changes in [$z_{0m}$] and [$z_{0h}$] by one order of magnitude can result in differences in the turbulent heat fluxes by a factor of two (Munro, 1989; Hock and Holmgren, 1996) demonstrating the significance of accurate roughness length determination. The relationship between the roughness lengths of wind, heat and vapour pressure is another matter of discussion. The principle of similarity is often invoked (e.g., Streten and Wendler, 1968; Hay and Fitzharris, 1988; Brun et al., 1989; Munro, 1989; Zuo and Oerlemans, 1996), although there is evidence that the surface roughnesses for heat and vapour pressure are smaller than [$z_{0m}$] by one or two orders of magnitude (Sverdrup, 1935; Holmgren, 1971; Ambach, 1986; Beljaars and Holtslag, 1991; Smeets et al., 1998). Conversely, Morris et al. (1994) concluded the opposite from energy-balance modelling, namely that [$z_{0h}$] and [$z_{0v}$] were considerably larger than the aerodynamic roughness
lengths. Data analysis from Greenland gave values for [$z_{0h}$] of about 10 to 100 times larger than [$z_{0m}$] (Calanca, 2001). Braithwaite (1995a) suggests that the assumption of unequal roughness lengths is not strictly necessary, because an ‘effective roughness length’ satisfying  [$z_{0m} \approx z_{0v} \approx z_{0h}$] can be chosen so that the exchange coefficient gives the same value when applying equations 24 and 25. The roughness length of vapour pressure is
generally assumed to be equal to the one for heat. However, very few studies, exist due to the inherent difficulty of accurately measuring vapour pressure profiles over glaciers. In addition, roughness lengths will vary in space and time (Plüss and Mazzoni, 1994; Greuell and Konzelmann, 1994; Calanca, 2001). Holmgren (1971) observed an increase
in roughness lengths with decreasing wind speed; Anderson (1976) a decrease with time during the snow melt season.*

An extensive overview of literature values for $z_{0m}$ is also given in Brock et al. (2006). For low- and midlatitude glaciers, a range from 0.1 to 50 mm is listed.

Alltogether, in the literature the roughness lengths are assumed to vary a lot, based on surface type (snow/ice), latitude, period of the year (accumulation/melting) and other factors.

For sensitivity experiments these values will be evaluated in the following:
- "normal" case
    - $z_{0m} = 1.7 \cdot 10^{-3} m$
    - $z_{0h} = z_{0v} = 1.2 \cdot 10^{-3} m$
- small roughness case
    - $z_{0m,s} = 5.6 \cdot 10^{-5} m$
    - $z_{0h,s} = z_{0v,s} = 6 \cdot 10^{-6} m$
- large roughness case
    - $z_{0m,l} = 0.05 m$
    - $z_{0h,l} = z_{0v,l} = 0.051 m$

Using the formula of the turbulent fluxes from above, the relative changes $Q_H$ and $Q_L$ may easily be computed (using the absolute measurement heights above ground, not disturbed by snowfall)

$Q_{H,s} = \frac{ln\left(\frac{z_m}{z_{0m}}\right) ln\left(\frac{z_h}{z_{0h}}\right)}{ln\left(\frac{z_m}{z_{0m,s}}\right) ln\left(\frac{z_h}{z_{0h,s}}\right)} Q_{H} \approx 0.4 \cdot Q_H$


$Q_{H,l} = \frac{ln\left(\frac{z_m}{z_{0m}}\right) ln\left(\frac{z_h}{z_{0h}}\right)}{ln\left(\frac{z_m}{z_{0m,l}}\right) ln\left(\frac{z_h}{z_{0h,l}}\right)} Q_{H} \approx 3.8 \cdot Q_H$


$Q_{L,s} = \frac{ln\left(\frac{z_m}{z_{0m}}\right) ln\left(\frac{z_v}{z_{0v}}\right)}{ln\left(\frac{z_m}{z_{0m,s}}\right) ln\left(\frac{z_v}{z_{0v,s}}\right)} Q_{L} \approx 0.4 \cdot Q_L$


$Q_{L,l} = \frac{ln\left(\frac{z_m}{z_{0m}}\right) ln\left(\frac{z_v}{z_{0v}}\right)}{ln\left(\frac{z_m}{z_{0m,l}}\right) ln\left(\frac{z_v}{z_{0v,l}}\right)} Q_{L} \approx 3.8 \cdot Q_L$

In [ ]:
z_0ms = 5.6e-5
z_0hs = 6e-6
z_0vs = 6e-6

z_0ml = 0.05
z_0hl = 0.051
z_0vl = 0.051

#note that z_0h = z_0v for all cases

factor_hs = np.log(z_m/z_0m) * np.log(z_h/z_0h) / (np.log(z_m/z_0ms) * np.log(z_h/z_0hs))
print(factor_hs)

factor_hl = np.log(z_m/z_0m) * np.log(z_h/z_0h) / (np.log(z_m/z_0ml) * np.log(z_h/z_0hl))
print(factor_hl)

To conclude, the turbulent fluxes may change strongly based on the selection of the roughness lengths. The value of turbulent heat fluxes can vary - while still using reasonable roughness lengths - between almost four times or less than half of what is given by the initially assumed roughness lengths.

As seen often in boundary layer meteorology, the turbulent fluxes are a big source of uncertainty and approximations can only give a rough estimate.

In [ ]:
df13['Q_H'].plot(label='$Q_H$');
df13['Q_L'].plot(label='$Q_L$');
(df13['Q_H'] * factor_hs).plot(label='$Q_{H, s}$');
(df13['Q_L'] * factor_hs).plot(label='$Q_{L, s}$');
(df13['Q_H'] * factor_hl).plot(label='$Q_{H, l}$');
(df13['Q_L'] * factor_hl).plot(label='$Q_{L, l}$');
#df13['NETRAD'].plot(label='net radiation');

plt.xlabel('');
plt.ylabel('Flux $\mathrm{(W m^{-2})}$');
plt.title('Sensible and latent heat flux for different roughness lengths - monthly averages');
plt.legend(loc="best");
h = plt.hlines(0, xmin=df11.index.min(), xmax=df11.index.max(), colors='k', linestyles='--', linewidth=0.5)

## 3 Residual Flux $F$ and melt energy $Q_M$

### 3.1 First look at residual flux

Neglecting the rain heat flux and the ground heat flux the residual flux $F$ is:
$$F=Q_\mathrm{net} + Q_H+Q_L$$

In [ ]:
df=df11.copy()
df["F"]=df.NETRAD+df.Q_H+df.Q_L
F_monthly=df["F"].resample("MS").mean()
F_monthly.plot()
plt.ylabel("Residual flux $\mathrm{(W m^{-2})}$")
plt.xlabel("")
plt.title("Monthly averaged residual flux")
plt.grid()

In summer the glacier surface is gaining more energy than it looses in winter. Both periods last about half a year.

In [ ]:
for i in [[1,"January"],[7,"July"]]:
    F_sel=df["F"].loc[df.index.month==i[0]]
    F_dcycle=F_sel.groupby(F_sel.index.hour).mean()
    F_dcycle.plot(label=i[1])
plt.legend()
plt.title("Monthly averaged diurnal cycle of the residual flux in January and July")
plt.xlabel("Hour")
plt.ylabel("Residual flux $\mathrm{(W m^{-2})}$")
plt.grid()

In July the surface is gaining large amounts of energy during the day mostly due to the net radiation. During the night the outgoing longwave radiation slightly overbalances the positive sensible heat flux.

In January negative sensible and latent heat fluxes lead to an energy loss during the day. During most of the night the outgoing longwave radiation leads to an energy loss, only during the early morning hours the positive sensible heat flux overbalances the outgoing longwave radiation.


### 3.2 The meaning of the residual flux

If we were not over a glacier surface, the residual flux as an energy gain or loss of the surface would directly lead to a temperature increase or decrease. For negative $F$ or temperatures below 0°C this is also true over a glacier surface. However, if the surface is gaining energy ($F>0$) while it is at freezing point, the residual flux leads to melting rather than to a temperature increase. The amount of melt is determined by the latent heat of melt $L_m$.


### 3.3 Energy available for melting 

We can compute the energy available for melting, $Q_m$, by taking the values of $F$, where the surface temperature is 0°C and $F>0$. The other values are set to zero.

In [ ]:
df["Qm"]=df["F"]
df["Qm"].loc[(df["SURFTEMP"]<0) | (df["F"]<=0)]=0



In [ ]:
df.Qm.plot()
plt.xlabel("")
plt.ylabel("$Q_m \: \mathrm{(W m^{-2})}$")
plt.title("Energy available for melting");

### 3.4 Averaging $Q_m$ and ranking the energy fluxes

In [ ]:
df["NETRAD"][df["Qm"]>0].mean(), df["Q_H"][df["Qm"]>0].mean(), df["Q_L"][df["Qm"]>0].mean()

The net radiation has by far the highest influence. The latent heat flux is on average decreasing the energy available for melting. It is more important for the energy balance of a melting glacier surface than the sensible heat flux.

TO DO: is this what he meant??

## 4 Mass balance

### 4.1 Compute the melt

In [ ]:
df["MELT"] = df["Qm"] / L_m #Melt in kg m⁻² s⁻¹
#Time step is one hour, precipitation is given in mm hr⁻¹ -> try to convert MELT to this unit
df["MELT"] *= 3600 #Melt in kg m⁻² hr⁻¹
#1kg w. e. approximately equals a volume of 1m x 1m x 1mm -> no more conversion needed for mm hr⁻¹

In [ ]:
df["MELT"].plot()
plt.title('Melt at the glacier surface')
plt.ylabel('Melt (mm w.e. hr$^{-1}$)')
plt.xlabel('');

As one would expect, the highest mass loss by melting can be observed during the summer time, when solar insolation and air temperatures are the highest.

### 4.2 Compute deposition and sublimation

Sublimation -> from solid directly to gas phase

Deposition -> from gas phase directly to solid

It is considered, that the energy loss/gain by these processes (+evaporation/condensation) equals the turbulent latent heat flux. Therefore:
$$Q_{subl, evap} =
    \begin{cases}
      Q_L, & \text{if}\ Q_L \le 0 \\
      0,  & \text{otherwise}
    \end{cases}$$

$$Q_{dep, cond} =
    \begin{cases}
      Q_L, & \text{if}\ Q_L > 0 \\
      0, & \text{otherwise}
    \end{cases}$$
    
For the mass balance, we are interested in the mm w.e. hr$^{-1}$. This calculates from every of the just defined energy fluxes by
$$MASS\_CHANGE = \frac{Q}{L} \cdot 3600$$
$L$ is hereby the latent heat of evaporation or sublimation and is defined as given in the project description and used in section 2. Note, that here also an additional minus sign may be necessary, to match the definitions. Fluxes are positive, if the ground gains energy, all changes in mass balance are defined positive and only connected by minus signs.

In [ ]:
L = get_L(df['TEMP_K'])

In [ ]:
#positive values indicate mass loss -> sublimation
df['SUB'] = - df['Q_L'] / L * 3600 #sublimation in mm w.e. hr⁻¹
#It is only sublimation if Q_L <= 0
df['SUB'].loc[df['Q_L'] > 0] = 0

In [ ]:
#positive values indicate mass gain -> deposition
df['DEP'] = df['Q_L'] / L * 3600 #deposition in mm w.e. hr⁻¹
#It is only sublimation if Q_L <= 0
df['DEP'].loc[df['Q_L'] <= 0] = 0

In [ ]:
df['SUB'].plot()
plt.title('Sublimation and evaporation at the glacier surface')
plt.ylabel('Sublimation & evaporation (mm w.e. hr$^{-1}$)')
plt.xlabel('');

In [ ]:
df['DEP'].plot()
plt.title('Deposition and condensation at the glacier surface')
plt.ylabel('Deposition & condensation (mm w.e. hr$^{-1}$)')
plt.xlabel('');

### 4.3 Simple solution for snowfall

Snow falls approximately if there is precipitation and the temperatures are low enough i.e. at 0°C or lower. The amount of snow fall can be estimated by its water equivalent, which should correspond to the measured precipitation. Note the missing precipitation (and therefore snowfall) values during the period 2011-09-16 00:00:00 to 2012-04-30 23:00:00.

<span class="mark">TO DO:</span> 
- Do we also need to estimate snowfall amount in cm? -> use a typical density

- Snow falls when the air temperature is <=0°C not the glacier surface (This would be always the case)!
Actually you usually also have snow fall at 1 or 2 °C since the time the snow falls through the warmer near surface layer is too short for the snow to melt, but it's probably enough to take 0°C air temperature as  the threshold


In [ ]:
df["SNOWFALL"] = df["PRCP"]
df["SNOWFALL"].loc[df["TEMP"] > 0] = 0

In [ ]:
df["SNOWFALL"].plot()
plt.title('Approximated w.e. of snowfall')
plt.ylabel('Snowfall (mm w.e. hr$^{-1}$)')
plt.xlabel('');

### 4.4 Approximation of the total mass balance (MB)

The computation of the total mass balance (approx.) now is only combining the precalculated values. The only thing to consider is the negative or positive definition of the values:
- snowfall is a positive value in case of mass gain
- melt is a positive value in case of mass loss -> subtract
- sublimation is a positive value in case of mass loss -> subtract
- deposition is a positive value in case of mass gain -> add

In [ ]:
df['MB'] = df['SNOWFALL'] + df['DEP'] - df['SUB'] - df['MELT']

In [ ]:
#Where are NAN values in Precipitation (and therefore the complete mass balance)?
nan_ind = df.loc[np.isnan(df['PRCP'])].index
print(nan_ind[0])
print(nan_ind[-1])
print(len(nan_ind) == 24 * (14 + 31 + 30 + 31 + 31+ 30 + 31 + 30))
print(24 * (14 + 31 + 30 + 31 + 31+ 30 + 31 + 30)) #theoretical number of nans for this period
print(len(nan_ind))

In [ ]:
#in PRCP there are NAN values contained -> for these times, no realistic mass balance can be found
#for plotting, all the other important values are NAN'ed out as well
df2 = df.copy()
df2['MELT'].loc[np.isnan(df2['PRCP'])] = np.NaN
df2['SUB'].loc[np.isnan(df2['PRCP'])] = np.NaN
df2['DEP'].loc[np.isnan(df2['PRCP'])] = np.NaN
df2['SNOWFALL'].loc[np.isnan(df2['PRCP'])] = np.NaN
# TODO: remove, not used anymore

In [ ]:

#TODO colors, monthly averages
df44 = df.resample('3D').mean()
ax = df44["MB"].plot(label="total MB")
ax = (- df44["MELT"]).plot(ax=ax, label="melt")
ax = df44["SNOWFALL"].plot(ax=ax, label="snowfall")
ax = (- df44['SUB']).plot(ax=ax, label="sublimation and evaporation")
ax = (- df44['DEP']).plot(ax=ax, label="deposition and condensation")
plt.legend(loc="best");
plt.ylabel('mass change (mm w.e. hr$^{-1}$)')
plt.xlabel('')
plt.title('Mass change in measurement period by component - 3 day average');
plt.tick_params(labelright=True, right=True);

In [ ]:
df45 = df.cumsum() / 1000 #mass changes are now in m w.e.

In [ ]:
ax = df45["MB"].plot(label="total MB")
ax = (- df45["MELT"]).plot(ax=ax, label="melt")
ax = df45["SNOWFALL"].plot(ax=ax, label="snowfall")
ax = (- df45['SUB']).plot(ax=ax, label="sublimation and evaporation")
ax = (- df45['DEP']).plot(ax=ax, label="deposition and condensation")
plt.legend(loc="best");
plt.ylabel('Accumulated mass change (m w.e.)')
plt.xlabel('')
plt.title('Accumulated mass change for measurement period');
plt.tick_params(labelright=True, right=True);

In [ ]:
df['MB'].sum() / 1000 #total mass balance (m w.e. for the whole period)

As seen above, mainly melt and snowfall determine the mass balance of the glacier. One major issue with this mass balance are the missing precipitation values from 2011-09-16 00:00:00 to 2012-04-30 23:00:00. This makes the accurate computation of the total mass balance impossible. However, when comparing the snowfall values for this period from the year before, it may be, that during this period there was no significant snowfall (TODO: check with SR50). So assuming, that the contribution by snowfall during this period is near zero, leads to a total mass loss of about 6.25 m w.e. during the measurement period. With respect to the ongoing climate change and melting of glaciers, this seems to be in the range of a realistic value. (TODO: check literature)

## 5 An attempt of validation

For validation of the above computed amount of melt we look at the melting season 2012, from 2012-05-15 to 2012-08-31.
To convert w.e. melt to surface height change we use  $350$ and $900$ $\mathrm{kg\,m^{-3}}$ as the density for snow and ice, respectively (as in *Klok and Oerlemans* (2002)). We divide the mass balance (in mm w.e. h⁻¹ $\sim$ kg m⁻² h⁻¹ ) by the density to obtain the the surface height change in m h⁻¹. 

To distinguish between snow and ice we use the SR50 sensor. Its value is calibrated the way that positive values indicate a snow surface in the regarded melting season, whereas negative values indicate an ice surface. 

In [ ]:
snow_dens=350
ice_dens=900
df5=df.loc["2012-05-15":"2012-08-31"].copy()
df5["density"]=np.where(df5.SR50>0,snow_dens, ice_dens)
df5["sfc_change"]=df5.MB/df5["density"]
df5["sfc_change_cum"]=np.cumsum(df5.sfc_change)+df5.SR50[0]

In [ ]:
df5.sfc_change_cum.plot(label="Modeled surface height")
df5.SR50.plot(label="Measured surface height")
plt.legend(loc="best")
plt.grid()
plt.xlabel("")
plt.ylabel("Surface height [m]")
plt.title("Modelled and measured surface height with $ \\rho_{\mathrm{snow}}=350\, \mathrm{kg\,m^{-3}}$ and $\\rho_{\mathrm{ice}}=900\, \mathrm{kg\,m^{-3}}$");

The shape of the curves is very similar. The slopes change at the same times from faster decay to stagnation and so on. However, the modeled surface height decreases faster than the measured one, especially when the glacier is covered with snow. The modeled surface height is monotonously decreasing, whereas the measured one shows some short periods of increase due to snowfall.

The major unknown in our calculation is the snow density. According to *Cuffey and Paterson *(2010) it ranges from $50$ (new snow after falling) to $800$ $\mathrm{kg\,m^{-3}}$ (very wet snow and firn). Especially shortly before the surface of the glacier turns to bare ice in the beginning of June the density is probably significantly greater than the assumed $350$ $\mathrm{kg\,m^{-3}}$.

<span class="burk">TODO:</span> Other sources of mass loss and gain not yet taken into account (Ranked according to their plausibility?):
- Additional mass gain due to:
    - refreezing meltwater
    - freezing rain
    - windborne snow 
    - basal accumulation
    - avalanching (not so plausible, since no big sudden increases measured)


- Mass loss due to:
    - Rain heat flux
    - ground heat flux


Using higher densities of $700$ and $1000$ $\mathrm{kg\,m^{-3}}$ for snow and ice, respectively, results in a better agreement of modeled and measured surface height.

In [ ]:
snow_dens=700
ice_dens=1000
df5=df.loc["2012-05-15":"2012-08-31"].copy()
df5["density"]=np.where(df5.SR50>0,snow_dens, ice_dens)
df5["sfc_change"]=df5.MB/df5["density"]
df5["sfc_change_cum"]=np.cumsum(df5.sfc_change)+df5.SR50[0]

In [ ]:
df5.sfc_change_cum.plot(label="Modeled surface height")
df5.SR50.plot(label="Measured surface height")
plt.legend(loc="best")
plt.grid()
plt.xlabel("")
plt.ylabel("Surface height [m]")
plt.title("Modelled and measured surface height with $ \\rho_{\mathrm{snow}}=700\, \mathrm{kg\,m^{-3}}$ and $\\rho_{\mathrm{ice}}=1000\, \mathrm{kg\,m^{-3}}$");

Now the two curves match very well except during Mid-June and beginning to Mid-August.

<span class="burk">TO DO</span>: WHY?

## References


- Ambach, W. (1986) "Nomographs for the determination of meltwater from snow and ice surfaces."
- Brock, B. W., Willis, I. C., and Sharp, M. J. (2006). "Measurement and parameterization of aerodynamic roughness length variations at haut glacier d’arolla, switzerland." Journal of Glaciology, 52(177):281–297.
- Hock, R. (2005). "Glacier melt: a review of processes and their modelling. Progress in physical geography", 29(3):362–391.
- King, J. and Anderson, P. (1994). "Heat and water vapour fluxes and scalar roughness lengths over an antarctic ice shelf." Boundary-Layer Meteorology, 69(1):101–121.
- Maussion, Fabien, et al. "Glaciological field studies at Zhadang Glacier (5500–6095 m), Tibetan Plateau." Workshop on the use of automatic measuring systems on glaciers–Extended abstracts and recommendations of the IASC Workshop, edited by: Tijm-Reijmer, CH and Oerlemans, J. 2011.
- Klok, E. J., and Johannes Oerlemans. "Model study of the spatial distribution of the energy and mass balance of Morteratschgletscher, Switzerland." Journal of Glaciology 48.163 (2002): 505-518.
- Cuffey, Kurt M., and William Stanley Bryce Paterson. The physics of glaciers. Academic Press, 2010.